In [1]:
import pandas as pd
import numpy as np

In [2]:
conversations = pd.read_csv("../target_behaviour_identification.csv")

In [3]:
conversations.head()

,chat_id,combined_text,category,user_utterance_english,bot_utterance_english,condition
0,1.0,"Start Hi! I'm MIcha, your motivational chatbot...",less procrastination,Start I want to procrastinate less I could be ...,"Hi! I'm MIcha, your motivational chatbot. My g...",framework
1,2.0,"Start Hi! I'm MIcha, your motivational chatbot...",less procrastination,Start I want to procrastinate less There are i...,"Hi! I'm MIcha, your motivational chatbot. My g...",no_system
2,3.0,"Start Hi! I'm MIcha, your motivational chatbot...",less procrastination,Start I want to procrastinate less I'm unprodu...,"Hi! I'm MIcha, your motivational chatbot. My g...",no_system
3,4.0,"Start Hi! I'm MIcha, your motivational chatbot...",healthier eating,Start I want to eat healthier I want to change...,"Hi! I'm MIcha, your motivational chatbot. My g...",framework
4,5.0,"Start Hi! I'm MIcha, your motivational chatbot...",healthier eating,Start I want to eat healthier I am overweight ...,"Hi! I'm MIcha, your motivational chatbot. My g...",no_system


In [4]:
conversations['category'].value_counts()

category
less procrastination    99
healthier eating        49
sustainable living      12
Name: count, dtype: int64

In [5]:
conversations[conversations['condition'] == 'framework']['category'].value_counts()

category
less procrastination    52
healthier eating        27
sustainable living       2
Name: count, dtype: int64

In [6]:
conversations[conversations['condition'] == 'no_system']['category'].value_counts()

category
less procrastination    47
healthier eating        22
sustainable living      10
Name: count, dtype: int64

#### Take 40% rows out as our sample for topic generatoin

In [79]:
import pandas as pd

sample_cf = conversations.sample(frac=0.4, random_state=42)

## Sample topics

#### Label: stress
Definition: Refers to experiences of psychological tension, anxiety, or emotional strain, often caused by pressure, uncertainty, or personal challenges.

#### Label: energy
Definition: Involves discussions about physical vitality, fatigue, motivation, and factors influencing one’s ability to stay active and alert.

#### Label: sharing
Definition: Describes moments when individuals disclose personal experiences, thoughts, or challenges, often to build trust or seek support.


In [ ]:
import openai
import time

# initialise OpenAI 
client = openai.OpenAI(api_key="*** ***")

# function：create prompt
def build_prompt(dialogues):
    joined_dialogues = "\n\n---\n\n".join(dialogues)
    prompt = f"""You are a helpful assistant trained to extract high-level topics from text.

Below is a collection of user-AI dialogues. Your task is to generate a list of concise and distinct topics that summarize the themes or ideas discussed, I’m expecting you to identify around 10 to 20 distinct topics in each API request. I prefer topics that are specific, detailed, and granular rather than overly broad, give a short and general definition (one sentence).

The format should be:
TopicName: Definition

and here's some good examples you can refer:
TopicName: Stress
Definition: Refers to experiences of psychological tension, anxiety, or emotional strain, often caused by pressure, uncertainty, or personal challenges.
TopicName: Energy
Definition: Involves discussions about physical vitality, fatigue, motivation, and factors influencing one’s ability to stay active and alert.
TopicName: Sharing
Definition: Describes moments when individuals disclose personal experiences, thoughts, or challenges, often to build trust or seek support.

Avoid redundancy. Focus on behavior change, goals, emotions, or common patterns in the dialogues.

Dialogues:
{joined_dialogues}

Now list the topics below:
"""
    return prompt
    
# loading data
dialogs = sample_cf["combined_text"].tolist()

# batch parameter
batch_size = 5
all_outputs = []

# iterate batch and send them to API
for i in range(0, len(dialogs), batch_size):
    batch = dialogs[i:i+batch_size]
    prompt = build_prompt(batch)

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        output = response.choices[0].message.content
        print(f"✅ Batch {i//batch_size + 1} complete.")
        all_outputs.append(output)
        time.sleep(5)

    except Exception as e:
        print(f"❌ Error in batch {i//batch_size + 1}: {e}")
        continue

# save result
with open("raw_generated_topics.txt", "w", encoding="utf-8") as f:
    f.write("\n\n=====\n\n".join(all_outputs))

print(f"Saved {len(all_outputs)} batch outputs to 'raw_generated_topics.txt'. Each batch contains topics generated from {batch_size} dialogues.")



✅ Batch 1 complete.
✅ Batch 2 complete.
✅ Batch 3 complete.
✅ Batch 4 complete.
